```
--- Day 17: Trick Shot ---
You finally decode the Elves' message. HI, the message says. You continue searching for the sleigh keys.

Ahead of you is what appears to be a large ocean trench. Could the keys have fallen into it? You'd better send a probe to investigate.

The probe launcher on your submarine can fire the probe with any integer velocity in the x (forward) and y (upward, or downward if negative) directions. For example, an initial x,y velocity like 0,10 would fire the probe straight up, while an initial velocity like 10,-1 would fire the probe forward at a slight downward angle.

The probe's x,y position starts at 0,0. Then, it will follow some trajectory by moving in steps. On each step, these changes occur in the following order:

The probe's x position increases by its x velocity.
The probe's y position increases by its y velocity.
Due to drag, the probe's x velocity changes by 1 toward the value 0; that is, it decreases by 1 if it is greater than 0, increases by 1 if it is less than 0, or does not change if it is already 0.
Due to gravity, the probe's y velocity decreases by 1.
For the probe to successfully make it into the trench, the probe must be on some trajectory that causes it to be within a target area after any step. The submarine computer has already calculated this target area (your puzzle input). For example:

target area: x=20..30, y=-10..-5
This target area means that you need to find initial x,y velocity values such that after any step, the probe's x position is at least 20 and at most 30, and the probe's y position is at least -10 and at most -5.

Given this target area, one initial velocity that causes the probe to be within the target area after any step is 7,2:

.............#....#............
.......#..............#........
...............................
S........................#.....
...............................
...............................
...........................#...
...............................
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTT#TT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
In this diagram, S is the probe's initial position, 0,0. The x coordinate increases to the right, and the y coordinate increases upward. In the bottom right, positions that are within the target area are shown as T. After each step (until the target area is reached), the position of the probe is marked with #. (The bottom-right # is both a position the probe reaches and a position in the target area.)

Another initial velocity that causes the probe to be within the target area after any step is 6,3:

...............#..#............
...........#........#..........
...............................
......#..............#.........
...............................
...............................
S....................#.........
...............................
...............................
...............................
.....................#.........
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................T#TTTTTTTTT
....................TTTTTTTTTTT
Another one is 9,0:

S........#.....................
.................#.............
...............................
........................#......
...............................
....................TTTTTTTTTTT
....................TTTTTTTTTT#
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
....................TTTTTTTTTTT
One initial velocity that doesn't cause the probe to be within the target area after any step is 17,-4:

S..............................................................
...............................................................
...............................................................
...............................................................
.................#.............................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT................................
....................TTTTTTTTTTT..#.............................
....................TTTTTTTTTTT................................
...............................................................
...............................................................
...............................................................
...............................................................
................................................#..............
...............................................................
...............................................................
...............................................................
...............................................................
...............................................................
...............................................................
..............................................................#
The probe appears to pass through the target area, but is never within it after any step. Instead, it continues down and to the right - only the first few steps are shown.

If you're going to fire a highly scientific probe out of a super cool probe launcher, you might as well do it with style. How high can you make the probe go while still reaching the target area?

In the above example, using an initial velocity of 6,9 is the best you can do, causing the probe to reach a maximum y position of 45. (Any higher initial y velocity causes the probe to overshoot the target area entirely.)

Find the initial velocity that causes the probe to reach the highest y position and still eventually be within the target area after any step. What is the highest y position it reaches on this trajectory?

```

In [1]:
import numpy as np

In [2]:
def step(loc, vel):
    loc[0] += vel[0]
    loc[1] += vel[1]
    if vel[0]>0:
        vel[0] -=1
    elif vel[0]<0:
        vel[0] +=1
    vel[1] -=1
    return loc, vel

In [3]:
def check_within(loc, xr,yr):
    if (xr[0]<=loc[0]<=xr[1]) and (yr[0]<=loc[1]<=yr[1]):
        return True
    else:
        return False

In [4]:
def load_target(infile):
    data = open(infile).read()[:-1]
    area = data.split(':')[-1]
    x,y = area.split(',')
    x_range = x.split('=')[-1].strip()
    x_range = [int(x) for x in x_range.split('..')]
    #x_range = slice(x_range[0], x_range[1]+2)
    
    y_range = y.split('=')[-1].strip()
    y_range = [int(y) for y in y_range.split('..')]
    #y_range = slice(y_range[0], y_range[1]+2)
    return x_range, y_range

In [5]:
def hits_target(loc, vel, xr, yr):
    highest = 0
    while (loc[1] > yr[0]) and (loc[0] < xr[1]):
        loc, vel = step(loc, vel)
        highest = max(loc[1],highest)
        #print(loc, vel)
        if check_within(loc, xr, yr):
            #print("bingo")
            return True, highest
    return False, highest

In [6]:
def solution1(infile):
    xr, yr =load_target(infile)
    best_vel = None
    highest = 0
    xmx = max(xr)+1
    ymx = max(np.abs(yr))+1
    for x in range(-xmx,xmx ):
        for y in range(0-ymx, ymx ):
            hit, height = hits_target([0,0], [x,y], xr, yr)
            #print(x,y, hit, height)
            if hit and (height>highest):
                highest=height
                best_vel = [x,y]
    print(f"Greatest height {highest} reached for v={best_vel}")

In [7]:
solution1('input.txt')

Greatest height 7750 reached for v=[17, 124]


```
--- Part Two ---
Maybe a fancy trick shot isn't the best idea; after all, you only have one probe, so you had better not miss.

To get the best idea of what your options are for launching the probe, you need to find every initial velocity that causes the probe to eventually be within the target area after any step.

In the above example, there are 112 different initial velocity values that meet these criteria:

23,-10  25,-9   27,-5   29,-6   22,-6   21,-7   9,0     27,-7   24,-5
25,-7   26,-6   25,-5   6,8     11,-2   20,-5   29,-10  6,3     28,-7
8,0     30,-6   29,-8   20,-10  6,7     6,4     6,1     14,-4   21,-6
26,-10  7,-1    7,7     8,-1    21,-9   6,2     20,-7   30,-10  14,-3
20,-8   13,-2   7,3     28,-8   29,-9   15,-3   22,-5   26,-8   25,-8
25,-6   15,-4   9,-2    15,-2   12,-2   28,-9   12,-3   24,-6   23,-7
25,-10  7,8     11,-3   26,-7   7,1     23,-9   6,0     22,-10  27,-6
8,1     22,-8   13,-4   7,6     28,-6   11,-4   12,-4   26,-9   7,4
24,-10  23,-8   30,-8   7,0     9,-1    10,-1   26,-5   22,-9   6,5
7,5     23,-6   28,-10  10,-2   11,-1   20,-9   14,-2   29,-7   13,-3
23,-5   24,-8   27,-9   30,-7   28,-5   21,-10  7,9     6,6     21,-5
27,-10  7,2     30,-9   21,-8   22,-7   24,-9   20,-6   6,9     29,-5
8,-2    27,-8   30,-5   24,-7
How many distinct initial velocity values cause the probe to be within the target area after any step?
```

In [10]:
def solution2(infile):
    xr, yr =load_target(infile)
    hit_list = []
    xmx = max(xr)+1
    ymx = max(np.abs(yr))+1
    for x in range(-xmx,xmx ):
        for y in range(0-ymx, ymx ):
            hit, height = hits_target([0,0], [x,y], xr, yr)
            if hit:
                hit_list.append([x,y])
    print(f"There are {len(hit_list)} values that will hit the target")
    

In [11]:

solution2('ex1.txt')

There are 112 values that will hit the target


In [12]:
solution2('input.txt')

There are 4120 values that will hit the target
